In [1]:
import numpy
import os
import pandas

from datetime import datetime
from sklearn.model_selection import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *

C:\Users\yamia\AppData\Local\Temp\ipykernel_3432\1888600386.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
df = pandas.read_csv('animal_chess.csv')

In [3]:
df = df[:100000]

In [4]:
count = len(df)

In [5]:
sample = df.iloc[32]
sample

board     ----r-E-Tl-----WC--d---------p------L--R--P---...
side                                                     -1
piece                                                     L
atk                                                       0
move                                                   F1E1
river                                                     0
trap                                                      1
den                                                       0
score                                                   -90
winner                                                    0
Name: 32, dtype: object

In [6]:
# Encode the piece
def encode_piece(piece_char):
    piece_mapping = {'-': 0, 'r': 1, 'c': 2, 'd': 3, 'w': 4, 'p': 5, 't': 6, 'l': 7, 'e': 8, 'R': -1, 'C': -2, 'D': -3, 'W': -4, 'P': -5, 'T': -6, 'L': -7, 'E': -8}
    return piece_mapping.get(piece_char, 0)

encode_piece(sample['piece'])

-7

In [7]:
# Encode the board
def encode_board(board_str):
    board_matrix = numpy.zeros((9, 7))
    for i, piece in enumerate(board_str[::-1]):
        row, col = divmod(i, 9)
        board_matrix[col][row] = encode_piece(piece)
    return numpy.flip(numpy.flip(board_matrix, 0), 1)

encode_board(sample['board'])

array([[ 0.,  7.,  0.,  0., -7.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  5.,  0.,  0.,  4.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-8., -4.,  0.,  0., -5.,  0.,  0.],
       [ 0., -2.,  0.,  0.,  0., -3.,  0.],
       [-6.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [8]:
df['board_encoded'] = df['board'].apply(encode_board)
board_matrix_flattened = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
board_matrix_flattened

array([[ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       ...,
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.]])

In [9]:
X = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
y = df['score'].values

In [12]:
# Split the data
def build_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(0.0001), loss='mean_squared_error', metrics=['mae'])
    return model

model = build_model((9, 7, 1))
model.summary()

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir=os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S")), histogram_freq=1)
checkpoint = ModelCheckpoint("best_model_0.h5", monitor='val_loss', save_best_only=True)

results = []

for train_index, test_index in KFold(n_splits=5).split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    history = model.fit(X_train.reshape(-1, 9, 7, 1), y_train, epochs=50, batch_size=512, validation_data=(X_test.reshape(-1, 9, 7, 1), y_test), callbacks=[reduce_lr, early_stopping, tensorboard_callback, checkpoint])
    results.append(model.evaluate(X_test.reshape(-1, 9, 7, 1), y_test))

print("Fold results:", results)
print("Average result:", numpy.mean(results, axis=0))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 9, 7, 64)          640       
                                                                 
 batch_normalization_4 (Bat  (None, 9, 7, 64)          256       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 4, 3, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 3, 128)         73856     
                                                                 
 batch_normalization_5 (Bat  (None, 4, 3, 128)         512       
 chNormalization)                                                
                                                      